In [6]:
import numpy as np
import pandas as pd

# Generate mock data for 100 students with real names and 4 ranked choices for each session
students_data = {
    'student_id': range(1, 101),
    'name': np.random.choice(
        ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Frank', 'Grace', 'Hannah', 'Ivy', 'Jack',
         'Kara', 'Liam', 'Mia', 'Nina', 'Oscar', 'Paul', 'Quinn', 'Rachel', 'Sam', 'Tina',
         'Uma', 'Victor', 'Wendy', 'Xander', 'Yara', 'Zach', 'Bella', 'Chris', 'Diana', 'Ethan',
         'Fiona', 'George', 'Holly', 'Ian', 'Jill', 'Kyle', 'Laura', 'Matt', 'Nate', 'Olivia',
         'Pete', 'Queen', 'Ryan', 'Sara', 'Tom', 'Ursula', 'Vince', 'Will', 'Xena', 'Yvonne',
         'Zane', 'Alyssa', 'Brian', 'Cathy', 'Derek', 'Emma', 'Felix', 'Gloria', 'Harry', 'Iris',
         'Jake', 'Kelly', 'Leo', 'Mona', 'Nick', 'Owen', 'Piper', 'Quentin', 'Rose', 'Steve', 
         'Tara', 'Ulysses', 'Violet', 'Wade', 'Ximena', 'Yusuf', 'Zelda', 'Amber', 'Brad', 'Cindy', 
         'Dean', 'Ella', 'Franklin', 'Gina', 'Hector', 'Isla', 'Jon', 'Kate', 'Louis', 'Maggie', 
         'Nolan', 'Opal', 'Paula', 'Quincy', 'Rob', 'Sophie', 'Trent', 'Uriah', 'Vera', 'Winston'],
        size=100, replace=False),
    'grade': np.random.randint(1, 9, size=100)
}

# Define the periods
periods = ['Monday AM', 'Monday PM', 'Tuesday AM', 'Tuesday PM', 'Wednesday AM', 'Wednesday PM', 'Thursday AM', 'Thursday PM', 'Friday AM', 'Friday PM']

# Add 4 ranked choices for each period
for period in periods:
    for choice in range(1, 5):
        students_data[f'choice{choice}_{period.replace(" ", "_").lower()}'] = np.random.randint(1, 21, size=100)

students_df = pd.DataFrame(students_data)

# Generate mock data for 20 activities with real names and sessions
activity_names = [
    'Art', 'Science', 'Math Games', 'Robotics', 'Music', 'Sports', 
    'Painting', 'Cooking', 'Reading', 'Drama', 'Chess', 'Football', 
    'Rock Climbing', 'Dance', 'Photography', 'Coding', 'Gardening', 'Writing', 
    'Astronomy', 'Yoga'
]

activities_data = {
    'activity_id': range(1, 21),
    'name': activity_names,
    'min_grade': np.random.randint(1, 5, size=20),
    'max_grade': np.random.randint(4, 9, size=20),
    'max_capacity': np.random.randint(5, 15, size=20),
    'sessions': np.random.randint(1, 5, size=20),
    'available_sessions': [",".join(np.random.choice(periods, size=np.random.randint(1, 6), replace=False)) for _ in range(20)]
}

activities_df = pd.DataFrame(activities_data)

# Generate mock data for 30 conflicts
conflicts_data = {
    'student_id1': np.random.randint(1, 101, size=30),
    'student_id2': np.random.randint(1, 101, size=30),
}

# Ensure no self-conflicts
conflicts_df = pd.DataFrame(conflicts_data)
conflicts_df = conflicts_df[conflicts_df['student_id1'] != conflicts_df['student_id2']]

# Generate mock data for group conflicts
group_conflicts_data = {
    'group_id': np.repeat(range(1, 11), 4),
    'student_id': np.random.randint(1, 101, size=40)
}
group_conflicts_df = pd.DataFrame(group_conflicts_data)

# Save data to CSV
students_df.to_csv('data/students.csv', index=False)
activities_df.to_csv('data/activities.csv', index=False)
conflicts_df.to_csv('data/conflicts.csv', index=False)
group_conflicts_df.to_csv('data/group_conflicts.csv', index=False)

students_df.head(), activities_df.head(), conflicts_df.head(), group_conflicts_df.head()


(   student_id   name  grade  choice1_monday_am  choice2_monday_am  \
 0           1  Paula      4                  9                 14   
 1           2  Chris      4                  7                 16   
 2           3  Kelly      6                 18                  9   
 3           4   Jill      2                 12                  8   
 4           5  Laura      7                 11                 14   
 
    choice3_monday_am  choice4_monday_am  choice1_monday_pm  choice2_monday_pm  \
 0                  5                 11                 13                 16   
 1                 12                 16                 12                 13   
 2                 17                  8                 13                 19   
 3                 13                 14                 20                  3   
 4                  5                  9                 16                 20   
 
    choice3_monday_pm  ...  choice3_thursday_pm  choice4_thursday_pm  \
 0          

In [15]:
import pandas as pd
from ortools.sat.python import cp_model

# Load data
students = pd.read_csv('data/students.csv')  # student_id, name, grade, choice1_monday_am, choice2_monday_am, ..., choice4_friday_pm
activities = pd.read_csv('data/activities.csv')  # activity_id, name, min_grade, max_grade, max_capacity, sessions, available_sessions
conflicts = pd.read_csv('data/conflicts.csv')  # student_id1, student_id2
group_conflicts = pd.read_csv('data/group_conflicts.csv')  # group_id, student_id

# Initialize the model
model = cp_model.CpModel()

# Create decision variables
assignments = {}  # key: (student_id, activity_id, period, session), value: decision variable

# Parse available sessions
sessions_per_activity = activities.set_index('activity_id')['sessions'].to_dict()
available_sessions = activities.set_index('activity_id')['available_sessions'].apply(lambda x: x.split(',')).to_dict()

# Periods definition
periods = ['monday_am', 'monday_pm', 'tuesday_am', 'tuesday_pm', 'wednesday_am', 'wednesday_pm', 'thursday_am', 'thursday_pm', 'friday_am', 'friday_pm']

for _, student in students.iterrows():
    student_id = student['student_id']
    grade = student['grade']
    for period in periods:
        for choice_rank in range(1, 5):
            activity_id = student[f'choice{choice_rank}_{period}']
            if activity_id in activities['activity_id'].values:
                activity = activities.loc[activities['activity_id'] == activity_id].iloc[0]
                if activity['min_grade'] <= grade <= activity['max_grade']:
                    # Determine the number of sessions for the activity
                    num_sessions = sessions_per_activity.get(activity_id, 1)
                    sessions = available_sessions.get(activity_id, [])
                    for session in range(min(num_sessions, len(sessions))):
                        if period.upper().replace('_', ' ') in sessions[session]:
                            assignments[(student_id, activity_id, period, session)] = model.NewBoolVar(f'assign_{student_id}_{activity_id}_{period}_session{session}')

# Add constraints
# Each student can be assigned to one activity per period
for _, student in students.iterrows():
    student_id = student['student_id']
    for period in periods:
        model.Add(sum(assignments[(student_id, activity_id, period, session)] for activity_id in activities['activity_id'].values for session in range(sessions_per_activity.get(activity_id, 1)) if (student_id, activity_id, period, session) in assignments) == 1)

# Activity capacity constraints
for _, activity in activities.iterrows():
    activity_id = activity['activity_id']
    max_capacity = activity['max_capacity']
    num_sessions = sessions_per_activity.get(activity_id, 1)
    for period in periods:
        for session in range(min(num_sessions, len(available_sessions.get(activity_id, [])))):
            if (activity_id, period, session) in available_sessions:
                model.Add(sum(assignments[(student_id, activity_id, period, session)] for student_id in students['student_id'].values if (student_id, activity_id, period, session) in assignments) <= max_capacity)

# Pairwise conflict constraints
for _, conflict in conflicts.iterrows():
    student_id1 = conflict['student_id1']
    student_id2 = conflict['student_id2']
    for period in periods:
        for activity_id in activities['activity_id'].values:
            num_sessions = sessions_per_activity.get(activity_id, 1)
            for session in range(min(num_sessions, len(available_sessions.get(activity_id, [])))):
                if (student_id1, activity_id, period, session) in assignments and (student_id2, activity_id, period, session) in assignments:
                    model.Add(assignments[(student_id1, activity_id, period, session)] + assignments[(student_id2, activity_id, period, session)] <= 1)

# Group conflict constraints
for group_id, group in group_conflicts.groupby('group_id'):
    students_in_group = group['student_id'].values
    for period in periods:
        for activity_id in activities['activity_id'].values:
            num_sessions = sessions_per_activity.get(activity_id, 1)
            for session in range(min(num_sessions, len(available_sessions.get(activity_id, [])))):
                if all((student_id, activity_id, period, session) in assignments for student_id in students_in_group):
                    model.Add(sum(assignments[(student_id, activity_id, period, session)] for student_id in students_in_group) <= len(students_in_group) - 1)

# Objective: Maximize the total satisfaction based on choices
objective_terms = []
for _, student in students.iterrows():
    student_id = student['student_id']
    for period in periods:
        for choice_rank in range(1, 5):
            activity_id = student[f'choice{choice_rank}_{period}']
            if activity_id in activities['activity_id'].values:
                num_sessions = sessions_per_activity.get(activity_id, 1)
                for session in range(min(num_sessions, len(available_sessions.get(activity_id, [])))):
                    if (student_id, activity_id, period, session) in assignments:
                        objective_terms.append((5 - choice_rank) * assignments[(student_id, activity_id, period, session)])

model.Maximize(sum(objective_terms))

# Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Extract results
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    allocations = []
    for (student_id, activity_id, period, session), var in assignments.items():
        if solver.BooleanValue(var):
            allocations.append((student_id, activity_id, period, session))

    allocation_df = pd.DataFrame(allocations, columns=['student_id', 'activity_id', 'period', 'session'])
    allocation_df.to_csv('data/allocations.csv', index=False)
    print("Allocations have been saved to allocations.csv")
else:
    print("No feasible solution found.")

No feasible solution found.


In [19]:
import pandas as pd
from ortools.sat.python import cp_model

# Load data
students = pd.read_csv('data/students.csv')  # student_id, name, grade, choice1_monday_am, choice2_monday_am, ..., choice4_friday_pm
activities = pd.read_csv('data/activities.csv')  # activity_id, name, min_grade, max_grade, max_capacity, sessions, available_sessions
conflicts = pd.read_csv('data/conflicts.csv')  # student_id1, student_id2
group_conflicts = pd.read_csv('data/group_conflicts.csv')  # group_id, student_id

# Initialize the model
model = cp_model.CpModel()

# Create decision variables
assignments = {}  # key: (student_id, activity_id, period, session), value: decision variable
activity_times = {}  # key: (activity_id, period), value: decision variable

# Parse available sessions
sessions_per_activity = activities.set_index('activity_id')['sessions'].to_dict()
available_sessions = activities.set_index('activity_id')['available_sessions'].apply(lambda x: x.split(',')).to_dict()

# Periods definition
periods = ['monday_am', 'monday_pm', 'tuesday_am', 'tuesday_pm', 'wednesday_am', 'wednesday_pm', 'thursday_am', 'thursday_pm', 'friday_am', 'friday_pm']

# Create decision variables for activity times
for _, activity in activities.iterrows():
    activity_id = activity['activity_id']
    for period in periods:
        activity_times[(activity_id, period)] = model.NewBoolVar(f'activity_time_{activity_id}_{period}')

# Ensure each activity is assigned to at most one period
for activity_id in activities['activity_id'].values:
    model.Add(sum(activity_times[(activity_id, period)] for period in periods) <= 1)

# Create decision variables for student assignments
for _, student in students.iterrows():
    student_id = student['student_id']
    grade = student['grade']
    for period in periods:
        for choice_rank in range(1, 5):
            activity_id = student[f'choice{choice_rank}_{period}']
            if activity_id in activities['activity_id'].values:
                activity = activities.loc[activities['activity_id'] == activity_id].iloc[0]
                if activity['min_grade'] <= grade <= activity['max_grade']:
                    num_sessions = sessions_per_activity.get(activity_id, 1)
                    for session in range(num_sessions):
                        assignments[(student_id, activity_id, period, session)] = model.NewBoolVar(f'assign_{student_id}_{activity_id}_{period}_session{session}')

# Add constraints for student assignments
for _, student in students.iterrows():
    student_id = student['student_id']
    for period in periods:
        model.Add(sum(assignments[(student_id, activity_id, period, session)] for activity_id in activities['activity_id'].values for session in range(sessions_per_activity.get(activity_id, 1)) if (student_id, activity_id, period, session) in assignments) <= 1)

# Ensure students are only assigned to activities during their available periods
for (student_id, activity_id, period, session), var in assignments.items():
    model.Add(var <= activity_times[(activity_id, period)])

# Activity capacity constraints
for _, activity in activities.iterrows():
    activity_id = activity['activity_id']
    max_capacity = activity['max_capacity']
    num_sessions = sessions_per_activity.get(activity_id, 1)
    for period in periods:
        for session in range(num_sessions):
            if (activity_id, period, session) in assignments:
                model.Add(sum(assignments[(student_id, activity_id, period, session)] for student_id in students['student_id'].values if (student_id, activity_id, period, session) in assignments) <= max_capacity)

# Pairwise conflict constraints (soft constraints)
conflict_penalties = []
for _, conflict in conflicts.iterrows():
    student_id1 = conflict['student_id1']
    student_id2 = conflict['student_id2']
    for period in periods:
        for activity_id in activities['activity_id'].values:
            num_sessions = sessions_per_activity.get(activity_id, 1)
            for session in range(num_sessions):
                if (student_id1, activity_id, period, session) in assignments and (student_id2, activity_id, period, session) in assignments:
                    conflict_penalty = model.NewBoolVar(f'conflict_{student_id1}_{student_id2}_{activity_id}_{period}_{session}')
                    model.AddMultiplicationEquality(conflict_penalty, [assignments[(student_id1, activity_id, period, session)], assignments[(student_id2, activity_id, period, session)]])
                    conflict_penalties.append(conflict_penalty)

# Group conflict constraints (soft constraints)
group_conflict_penalties = []
for group_id, group in group_conflicts.groupby('group_id'):
    students_in_group = group['student_id'].values
    for period in periods:
        for activity_id in activities['activity_id'].values:
            num_sessions = sessions_per_activity.get(activity_id, 1)
            for session in range(num_sessions):
                if all((student_id, activity_id, period, session) in assignments for student_id in students_in_group):
                    group_conflict_penalty = model.NewBoolVar(f'group_conflict_{group_id}_{activity_id}_{period}_{session}')
                    model.Add(group_conflict_penalty == sum(assignments[(student_id, activity_id, period, session)] for student_id in students_in_group) >= len(students_in_group))
                    group_conflict_penalties.append(group_conflict_penalty)

# Objective: Maximize the total satisfaction based on choices and minimize conflicts
objective_terms = []
for _, student in students.iterrows():
    student_id = student['student_id']
    for period in periods:
        for choice_rank in range(1, 5):
            activity_id = student[f'choice{choice_rank}_{period}']
            if activity_id in activities['activity_id'].values:
                num_sessions = sessions_per_activity.get(activity_id, 1)
                for session in range(num_sessions):
                    if (student_id, activity_id, period, session) in assignments:
                        objective_terms.append((5 - choice_rank) * assignments[(student_id, activity_id, period, session)])

# Add penalties for conflicts to the objective
for penalty in conflict_penalties:
    objective_terms.append(-penalty)

for penalty in group_conflict_penalties:
    objective_terms.append(-penalty)

model.Maximize(sum(objective_terms))

# Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Extract results
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    allocations = []
    for (student_id, activity_id, period, session), var in assignments.items():
        if solver.BooleanValue(var):
            allocations.append((student_id, activity_id, period, session))

    if allocations:
        # Merge with student and activity names for better readability
        allocation_df = pd.DataFrame(allocations, columns=['student_id', 'activity_id', 'period', 'session'])
        allocation_df = allocation_df.merge(students[['student_id', 'name']], on='student_id', how='left')
        allocation_df = allocation_df.merge(activities[['activity_id', 'name']], on='activity_id', how='left', suffixes=('_student', '_activity'))
        allocation_df = allocation_df[['student_id', 'name_student', 'activity_id', 'name_activity', 'period', 'session']]
        allocation_df.to_csv('data/allocations.csv', index=False)
        print("Allocations have been saved to allocations.csv")
    else:
        print("No assignments were made.")
else:
    print("No feasible solution found.")

# Debug: Check if decision variables are being created
print(f"Total assignments variables created: {len(assignments)}")
print(f"Total activity times variables created: {len(activity_times)}")

Allocations have been saved to allocations.csv
Total assignments variables created: 3765
Total activity times variables created: 200


In [23]:
import pandas as pd

# Load data
students = pd.read_csv('data/students.csv')  # student_id, name, grade, choice1_monday_am, choice2_monday_am, ..., choice4_friday_pm
activities = pd.read_csv('data/activities.csv')  # activity_id, name, min_grade, max_grade, max_capacity, sessions, available_sessions
allocations = pd.read_csv('data/allocations.csv')  # student_id, activity_id, period, session

# Rename columns for merging
students.rename(columns={'name': 'student_name'}, inplace=True)
activities.rename(columns={'name': 'activity_name'}, inplace=True)

# Merge with student and activity names for better readability
allocations = allocations.merge(students[['student_id', 'student_name']], on='student_id', how='left')
allocations = allocations.merge(activities[['activity_id', 'activity_name']], on='activity_id', how='left')

# Create a schedule DataFrame
schedule = []

for period in allocations['period'].unique():
    for session in allocations['session'].unique():
        session_data = allocations[(allocations['period'] == period) & (allocations['session'] == session)]
        if not session_data.empty:
            for activity_id in session_data['activity_id'].unique():
                activity_data = session_data[session_data['activity_id'] == activity_id]
                activity_name = activity_data['activity_name'].iloc[0]
                students_list = activity_data[['student_id', 'student_name']].values.tolist()
                schedule.append([period, session, activity_id, activity_name, students_list])

# Convert the schedule to a DataFrame for better formatting
schedule_df = pd.DataFrame(schedule, columns=['Period', 'Session', 'Activity ID', 'Activity Name', 'Students'])

# Save the schedule to a CSV file
schedule_df.to_csv('data/schedule.csv', index=False)

# Print the schedule for review
print(schedule_df)

print("Schedule has been saved to schedule.csv")

          Period  Session  Activity ID  Activity Name  \
0      monday_am        0           14          Dance   
1      monday_am        0           20           Yoga   
2      monday_am        2           14          Dance   
3    thursday_am        0           13  Rock Climbing   
4    thursday_am        1           13  Rock Climbing   
5    thursday_am        1            1            Art   
6      friday_am        0           18        Writing   
7      friday_am        0           12       Football   
8      friday_am        1           12       Football   
9      monday_pm        0            4       Robotics   
10     monday_pm        0           19      Astronomy   
11     monday_pm        3           19      Astronomy   
12    tuesday_am        0           17      Gardening   
13    tuesday_am        0            6         Sports   
14    tuesday_am        1            6         Sports   
15    tuesday_am        2            9        Reading   
16   thursday_pm        0      

In [ ]:
import pandas as pd
from ortools.sat.python import cp_model

# Load data
students = pd.read_csv('data/students.csv')  # student_id, name, grade, choice1_monday_am, choice2_monday_am, ..., choice4_friday_pm
activities = pd.read_csv('data/activities.csv')  # activity_id, name, min_grade, max_grade, max_capacity, sessions, available_sessions
conflicts = pd.read_csv('data/conflicts.csv')  # student_id1, student_id2
group_conflicts = pd.read_csv('data/group_conflicts.csv')  # group_id, student_id

# Initialize the model
model = cp_model.CpModel()

# Create decision variables
assignments = {}  # key: (student_id, activity_id, period, session), value: decision variable
activity_times = {}  # key: (activity_id, period), value: decision variable

# Parse available sessions
sessions_per_activity = activities.set_index('activity_id')['sessions'].to_dict()
available_sessions = activities.set_index('activity_id')['available_sessions'].apply(lambda x: x.split(',')).to_dict()

# Periods definition
periods = ['monday_am', 'monday_pm', 'tuesday_am', 'tuesday_pm', 'wednesday_am', 'wednesday_pm', 'thursday_am', 'thursday_pm', 'friday_am', 'friday_pm']

# Create decision variables for activity times
for _, activity in activities.iterrows():
    activity_id = activity['activity_id']
    for period in periods:
        activity_times[(activity_id, period)] = model.NewBoolVar(f'activity_time_{activity_id}_{period}')

# Ensure each activity is assigned to at most one period
for activity_id in activities['activity_id'].values:
    model.Add(sum(activity_times[(activity_id, period)] for period in periods) <= 1)

# Create decision variables for student assignments
for _, student in students.iterrows():
    student_id = student['student_id']
    grade = student['grade']
    for period in periods:
        for choice_rank in range(1, 5):
            activity_id = student[f'choice{choice_rank}_{period}']
            if activity_id in activities['activity_id'].values:
                activity = activities.loc[activities['activity_id'] == activity_id].iloc[0]
                if activity['min_grade'] <= grade <= activity['max_grade']:
                    num_sessions = sessions_per_activity.get(activity_id, 1)
                    for session in range(num_sessions):
                        assignments[(student_id, activity_id, period, session)] = model.NewBoolVar(f'assign_{student_id}_{activity_id}_{period}_session{session}')

# Add constraints for student assignments
for _, student in students.iterrows():
    student_id = student['student_id']
    for period in periods:
        model.Add(sum(assignments[(student_id, activity_id, period, session)] for activity_id in activities['activity_id'].values for session in range(sessions_per_activity.get(activity_id, 1)) if (student_id, activity_id, period, session) in assignments) <= 1)

# Ensure students are only assigned to activities during their available periods
for (student_id, activity_id, period, session), var in assignments.items():
    model.Add(var <= activity_times[(activity_id, period)])

# Activity capacity constraints
for _, activity in activities.iterrows():
    activity_id = activity['activity_id']
    max_capacity = activity['max_capacity']
    num_sessions = sessions_per_activity.get(activity_id, 1)
    for period in periods:
        for session in range(num_sessions):
            if (activity_id, period, session) in assignments:
                model.Add(sum(assignments[(student_id, activity_id, period, session)] for student_id in students['student_id'].values if (student_id, activity_id, period, session) in assignments) <= max_capacity)

# Pairwise conflict constraints (soft constraints)
conflict_penalties = []
for _, conflict in conflicts.iterrows():
    student_id1 = conflict['student_id1']
    student_id2 = conflict['student_id2']
    for period in periods:
        for activity_id in activities['activity_id'].values:
            num_sessions = sessions_per_activity.get(activity_id, 1)
            for session in range(num_sessions):
                if (student_id1, activity_id, period, session) in assignments and (student_id2, activity_id, period, session) in assignments:
                    conflict_penalty = model.NewBoolVar(f'conflict_{student_id1}_{student_id2}_{activity_id}_{period}_{session}')
                    model.Add(conflict_penalty == assignments[(student_id1, activity_id, period, session)] * assignments[(student_id2, activity_id, period, session)])
                    conflict_penalties.append(conflict_penalty)

# Group conflict constraints (soft constraints)
group_conflict_penalties = []
for group_id, group in group_conflicts.groupby('group_id'):
    students_in_group = group['student_id'].values
    for period in periods:
        for activity_id in activities['activity_id'].values:
            num_sessions = sessions_per_activity.get(activity_id, 1)
            for session in range(num_sessions):
                if all((student_id, activity_id, period, session) in assignments for student_id in students_in_group):
                    group_conflict_penalty = model.NewBoolVar(f'group_conflict_{group_id}_{activity_id}_{period}_{session}')
                    model.Add(group_conflict_penalty == sum(assignments[(student_id, activity_id, period, session)] for student_id in students_in_group) >= len(students_in_group))
                    group_conflict_penalties.append(group_conflict_penalty)

# Objective: Maximize the total satisfaction based on choices and minimize conflicts
objective_terms = []
for _, student in students.iterrows():
    student_id = student['student_id']
    for period in periods:
        for choice_rank in range(1, 5):
            activity_id = student[f'choice{choice_rank}_{period}']
            if activity_id in activities['activity_id'].values:
                num_sessions = sessions_per_activity.get(activity_id, 1)
                for session in range(num_sessions):
                    if (student_id, activity_id, period, session) in assignments:
                        objective_terms.append((5 - choice_rank) * assignments[(student_id, activity_id, period, session)])

# Add penalties for conflicts to the objective
for penalty in conflict_penalties:
    objective_terms.append(-penalty)

for penalty in group_conflict_penalties:
    objective_terms.append(-penalty)

model.Maximize(sum(objective_terms))

# Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Extract results
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    allocations = []
    for (student_id, activity_id, period, session), var in assignments.items():
        if solver.BooleanValue(var):
            allocations.append((student_id, activity_id, period, session))

    allocation_df = pd.DataFrame(allocations, columns=['student_id', 'activity_id', 'period', 'session'])
    allocation_df.to_csv('data/allocations.csv', index=False)
    print("Allocations have been saved to allocations.csv")
else:
    print("No feasible solution found.")